# Coinbase Transactions Summary & Visualization

This notebook loads your exported `data/transactions.json` file, summarizes your activity, and visualizes it using pandas and matplotlib.

In [ ]:
# Imports
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%matplotlib inline
sns.set(style="whitegrid")

In [ ]:
# Load the transactions
with open('data/transactions.json') as f:
    txs = json.load(f)

print(f"Loaded {len(txs)} transactions.")

In [ ]:
# Normalize to pandas DataFrame
df = pd.json_normalize(txs)

# Show sample
df.head()

In [ ]:
# Parse useful columns
df['created_at'] = pd.to_datetime(df['created_at'])
df['amount'] = df['amount.amount'].astype(float)
df['currency'] = df['amount.currency']
df['type'] = df['type']
df['account_name'] = df['account_name']

# If native_amount exists, parse USD value
if 'native_amount.amount' in df.columns and 'native_amount.currency' in df.columns:
    df['native_amount'] = df['native_amount.amount'].astype(float)
    df['native_currency'] = df['native_amount.currency']
else:
    df['native_amount'] = float('nan')

df.head()

## Summary Stats

In [ ]:
print("Transaction type counts:")
print(df['type'].value_counts())

print("\nTotal by asset:")
print(df.groupby('currency')['amount'].sum())

if not df['native_amount'].isna().all():
    print("\nTotal USD value:")
    print(df['native_amount'].sum())

## Visualizations

In [ ]:
# Transactions over time
plt.figure(figsize=(12,5))
df.set_index('created_at').resample('M').size().plot(kind='bar', color='skyblue')
plt.title('Number of Transactions per Month')
plt.ylabel('Transactions')
plt.xlabel('Month')
plt.tight_layout()
plt.show()

In [ ]:
# Transaction types
plt.figure(figsize=(6,6))
df['type'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90)
plt.title('Transaction Types')
plt.ylabel('')
plt.show()

In [ ]:
# Amounts by asset
plt.figure(figsize=(10,5))
df.groupby('currency')['amount'].sum().sort_values().plot(kind='barh')
plt.title('Total Amount by Asset')
plt.xlabel('Total')
plt.tight_layout()
plt.show()

In [ ]:
# (Optional) USD volume over time
if not df['native_amount'].isna().all():
    df.set_index('created_at').resample('M')['native_amount'].sum().plot(figsize=(12,5))
    plt.title('USD Transaction Volume per Month')
    plt.ylabel('USD Volume')
    plt.xlabel('Month')
    plt.tight_layout()
    plt.show()

You can further analyze by filtering `df` for specific types, assets, or time periods. 

Let me know if you want additional plots or breakdowns!